In [1]:
import ee
import geemap
import folium

# Define the country name
country_name = "France"

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Get the country boundaries
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', country_name)).geometry()

# Get the Sentinel-2 image collection for 2022
se2_2022 = ee.ImageCollection('COPERNICUS/S2').filterDate("2022-01-01","2022-12-31").filterBounds(aoi)

# Get the Sentinel-2 image collection for 2023
se2_2023 = ee.ImageCollection('COPERNICUS/S2').filterDate("2023-01-01","2023-12-31").filterBounds(aoi)

# Create a difference image
se2_2022_median = se2_2022.median().divide(10000)
se2_2023_median = se2_2023.median().divide(10000)
diff_image = se2_2023_median.subtract(se2_2022_median)

# Function to calculate NDVI for an image
def calculate_ndvi(image):
    # Extract the near-infrared (B8A) and red (B2) bands
    nir = image.select('B8A')
    red = image.select('B2')

    # Calculate the NDVI
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

    return image.addBands(ndvi)


# Map the NDVI calculation function to the image collections
se2_2022_with_ndvi = se2_2022.map(calculate_ndvi)
se2_2023_with_ndvi = se2_2023.map(calculate_ndvi)

# Calculate the difference in NDVI
se2_2022_median_with_ndvi = se2_2022_with_ndvi.median()
se2_2023_median_with_ndvi = se2_2023_with_ndvi.median()
diff_ndvi = se2_2023_median_with_ndvi.select('NDVI').subtract(se2_2022_median_with_ndvi.select('NDVI'))

# Create a folium Map
map1 = geemap.Map(center=[0, 0], zoom=2)

# Add the EE images to the map
map1.addLayer(diff_image, {"min": -0.5, "max": 0.5, "bands": ["B4", "B3", "B2"]}, "Difference Image")
map1.addLayer(diff_ndvi, {"min": -1, "max": 1, "palette": ['blue', 'white', 'green']}, "Difference NDVI")

# Display the map
map1


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=z-ubqu5u9ukSPo3kU-BteYC9DrN7EfALOjeZaCm5oOU&tc=pvCLnCIYyJoq6zDBambw2QnksGZIE48y04hBhgPH2S4&cc=Sy8w6kK_NprVXroSgFpMA_yGlU19dM-nygPupNAngeo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkoYySnPevEmCZqxmuZjthObpC1wI8ipcQu2te8s-f3C4afX57Z3aA

Successfully saved authorization token.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…